# Metoda računanja položaja

Namesto, da bi s pomočjo LOP položaj narisali ga lahko tudi izračunamo. Metoda je opisana v **Admirality Almanach**.

Imamo podan naš domnevni položaj **AP** ($\varphi_\text{AP},\lambda_\text{AP}$) in razliko v višini $\Delta h_i$ er azimut $\omega_i$ za vsako nebesno telo. S temi podatki lahko nato po naslednji metodi določimo računsko položaj.

Najprej poračunamo koeficiente $A_1,B_1,C_1,D_1,E_1,G_1$ s formulami

$$ A_1 = \sum_{i=1}^n \cos^2 \omega_i, ~~~ B_1 = \sum_{i=1}^n \sin \omega_i \: \cos \omega_i, ~~~ C_1 = \sum_{i=1}^n \sin^2 \omega_i, ~~~ D_1 = \sum_{i=1}^n \Delta h_i \: \cos \omega_i, ~~~ E_1 = \sum_{i=1}^n \Delta h_i \: \sin \omega_i, ~~~ G_1 = A_1 \cdot C_1 - B_1^2,$$

kjer je $i$ število nebesnih teles, ki smo jih imzmerili. Ko izračunamo koeficiente, lahko določimo položaj opazovlaca

$$ \varphi_1 = \varphi_\text{AP} + \frac{C_1 \cdot D_1 - B_1 \cdot E_1}{G_1}, ~~~~ \lambda_1 = \lambda_\text{AP} + \frac{A_1 \cdot E_1 - B_1 \cdot D_1}{G_1 \cdot \cos \varphi_\text{AP}}.$$

Sedaj lahko z novim položajem ($\varphi_1, \lambda_1$), ponovno izračunamo $\Delta h_i$ in $\omega_i$. Tako lahko ponovimo celoten postopek določitve koeficientov $A_2,B_2,C_2,D_2,E_2,G_2$ in izračunamo drugi novi položaj

$$ \varphi_2 = \varphi_1 + \frac{C_2 \cdot D_2 - B_2 \cdot E_2}{G_2}, ~~~~ \lambda_2 = \lambda_1 + \frac{A_2 \cdot E_2 - B_2 \cdot D_2}{G_2 \cdot \cos \varphi_1}.$$

To ponavljamo toliko časa, dokler se ($\varphi_k, \lambda_k$) in ($\varphi_{k+1}, \lambda_{k+1}$) ne razlikujeto poljubno natančno. 

Iteracijo v izračunu uvedemo zato, ker LOP ni premica ampak krožnica. Bližje kot sta računski položaj in pravi položaj manjša je napaka, te metode, ki temelji na domnevi, da je LOP premica.

<hr>

In [ ]:
import os, sys
import subprocess as sp

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

In [ ]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 7})

import celestialdata as cdata
import navigationalstars as ns
import navtools as nt

In [ ]:
def get_coefficients(dh,w,nn):

    A = 0
    for i in range(nn):
        A += mat.cos(nt.deg2rad(w[i]))**2

    B = 0
    for i in range(nn):
        B += mat.sin(nt.deg2rad(w[i]))*mat.cos(nt.deg2rad(w[i]))

    C = 0
    for i in range(nn):
        C += mat.sin(nt.deg2rad(w[i]))**2

    D = 0
    for i in range(nn):
        D += dh[i]*mat.cos(nt.deg2rad(w[i]))

    E = 0
    for i in range(nn):
        E += dh[i]*mat.sin(nt.deg2rad(w[i]))

    G = A*C - B**2

    return [A,B,C,D,E,G]

In [ ]:
def get_dh_array(Ho, Hc, nn, convert=False):

    dhA = []
    for i in range(nn):
        if convert:
            dh = nt.dms2dd(Ho[i]) - nt.dms2dd(Hc[i]) 
        else:
            dh = Ho[i] - Hc[i]
        dhA.append(dh)

    return np.array(dhA)

In [ ]:
def get_position(P,dh,w,nn):

    [A,B,C,D,E,G] = get_coefficients(dh,w,nn)
    
    fi = P[0] + (C*D - B*E)/G
    la = P[1] + (A*E - B*D)/(G * mat.cos(nt.deg2rad(P[0])))

    return [fi,la]

In [ ]:
# **************
# *** Primer ***
# **************

# Date: 2021/05/11, UT: 19:00:00
# Izmerimo: Alioth, Regulus in Elnath
# Sešteti položaj: 41 N, 20 W
# Pravi položaj: 41 30'N, 20 30'W (za kontrolo)

P = [41, -20]                        # assumed position
cbn = ['Alioth','Regulus','Elnath']  # imena nebesnih teles
Ho = [[49,38.2],[56,24.1],[45,57.1]] # izmerjene višine [deg, min]
Hc = [[49,35],[57,01.5],[45,34.1]]   # izračunane višine [deg, min]
w = [47.9, 147.1, 271.1]             # izračunani azimut
nn = 3                               # št. izmerjenih nebesnih teles

dh = get_dh_array(Ho,Hc,nn,True)     # izračunamo razliko v višini in rezultat izpišemo
print('razlika višine:')
for i in range(nn):
    print(' -> {:s}: {:.1f} min'.format(cbn[i],dh[i]*60))

[fi, la] = get_position(P,dh,w,nn)   # izračunamo novi položaj

print()
print(' AP: fi={:s}, la={:s}'.format(nt.prettyPrintLat(P[0]),nt.prettyPrintLong(P[1])))
print('    ------------------------------')
print('Fix: fi={:s}, la={:s}'.format(nt.prettyPrintLat(fi),nt.prettyPrintLong(la)))